In [7]:
import obj_2_pcd
import render_inference_result
import importlib
#import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import multiprocessing
#from vedo import dataurl, printc, Plotter, Points, Mesh, Text2D
import pytorch3d
import importlib
import obj_2_pcd
import render_inference_result
importlib.reload(obj_2_pcd)
importlib.reload(render_inference_result)
import torch
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")



import os
import shutil

data_ids = ['0408']
tiff_dir_root = '/data/jhahn/data/brain_lightsheet'
obj_dir_root = '/data/jhahn/data/shape_dataset/data/brain_lightsheet'
from_index=88
start_index=0
to_index=215
num_of_slices=20
tickness = 0.02



['magma', 'inferno', 'plasma', 'viridis', 'cividis', 'twilight', 'twilight_shifted', 'turbo', 'berlin', 'managua', 'vanimo', 'Blues', 'BrBG', 'BuGn', 'BuPu', 'CMRmap', 'GnBu', 'Greens', 'Greys', 'OrRd', 'Oranges', 'PRGn', 'PiYG', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy', 'RdPu', 'RdYlBu', 'RdYlGn', 'Reds', 'Spectral', 'Wistia', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd', 'afmhot', 'autumn', 'binary', 'bone', 'brg', 'bwr', 'cool', 'coolwarm', 'copper', 'cubehelix', 'flag', 'gist_earth', 'gist_gray', 'gist_heat', 'gist_ncar', 'gist_rainbow', 'gist_stern', 'gist_yarg', 'gnuplot', 'gnuplot2', 'gray', 'hot', 'hsv', 'jet', 'nipy_spectral', 'ocean', 'pink', 'prism', 'rainbow', 'seismic', 'spring', 'summer', 'terrain', 'winter', 'Accent', 'Dark2', 'Paired', 'Pastel1', 'Pastel2', 'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b', 'tab20c', 'grey', 'gist_grey', 'gist_yerg', 'Grays', 'magma_r', 'inferno_r', 'plasma_r', 'viridis_r', 'cividis_r', 'twilight_r', 'twilight_shifted_r', 't

In [4]:
dir = '/data/jhahn/data/shape_dataset/data/brain_lightsheet'
import os
for f in sorted(os.listdir(dir)):
    print('brain_lightsheet/'+f)

brain_lightsheet/0408
brain_lightsheet/0408_0088_215
brain_lightsheet/0408_0088_215_all
brain_lightsheet/0408_0088_215_random_rotation
brain_lightsheet/0408_0088_215_random_scale
brain_lightsheet/0408_0088_215_random_translation
brain_lightsheet/0408_0089_215
brain_lightsheet/0408_0089_215_all
brain_lightsheet/0408_0089_215_random_rotation
brain_lightsheet/0408_0089_215_random_scale
brain_lightsheet/0408_0089_215_random_translation
brain_lightsheet/0408_0090_215
brain_lightsheet/0408_0090_215_all
brain_lightsheet/0408_0090_215_random_rotation
brain_lightsheet/0408_0090_215_random_scale
brain_lightsheet/0408_0090_215_random_translation
brain_lightsheet/0408_0091_215
brain_lightsheet/0408_0091_215_all
brain_lightsheet/0408_0091_215_random_rotation
brain_lightsheet/0408_0091_215_random_scale
brain_lightsheet/0408_0091_215_random_translation
brain_lightsheet/0408_0092_215
brain_lightsheet/0408_0092_215_all
brain_lightsheet/0408_0092_215_random_rotation
brain_lightsheet/0408_0092_215_random

the number of jobs:1
/data/jhahn/data/shape_dataset/data/brain_lightsheet/0408_0088_215_random_rotation/fractured_0
